In [1]:
import json

from process_bigraph import Composite

from biosimulators_processes import CORE
from biosimulators_processes.steps.bio_compose import MongoDatabaseEmitter
from process_bigraph.composite import RAMEmitter
from biosimulators_processes.processes.copasi_process import CopasiProcess

In [2]:
uri = 'mongodb://localhost:27017/?retryWrites=true&w=majority&appName=biosimulators-processes'
emitter = MongoDatabaseEmitter(config={'connection_uri': uri}, core=CORE)
ram = RAMEmitter(config={}, core=CORE)
ram.inputs()

In [2]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

In [3]:
doc = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi-process',
        'config': {
            'model': {
                'model_source': model_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
            'model_parameters': ['model_parameters_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:database-emitter',
        'config': {
            'emit': {
                'time': 'float',
                'floating_species_concentrations': 'tree[float]'
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species_concentrations': ['floating_species_concentrations_store']
        }
    }
}


composite = Composite(config={'state': doc}, core=CORE)

composite.run(10)

In [4]:
composite.gather_results()

In [1]:
from abc import ABC, abstractmethod
from enum import Enum
from typing import *
from datetime import datetime

from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.database import Database


class DatabaseConnector(ABC):
    """Abstract class that is both serializable and interacts with the database (of any type). """
    def __init__(self, connection_uri: str, database_id: str, connector_id: str):
        self.database_id = database_id
        self.client = self._get_client(connection_uri)
        self.db = self._get_database(self.database_id)

    @staticmethod
    def timestamp() -> str:
        return str(datetime.utcnow())

    def refresh_data(self):
        def refresh_collection(coll):
            for job in self.db[coll].find():
                self.db[coll].delete_one(job)

        for collname in self.db.list_collection_names():
            refresh_collection(collname)

    @abstractmethod
    def _get_client(self, *args):
        pass

    @abstractmethod
    def _get_database(self, db_id: str):
        pass

    @abstractmethod
    async def read(self, *args, **kwargs):
        pass

    @abstractmethod
    async def write(self, *args, **kwargs):
        pass

    @abstractmethod
    def get_collection(self, **kwargs):
        pass


class JobStatus(Enum):
    PENDING = "PENDING"
    IN_PROGRESS = "IN_PROGRESS"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"


class DatabaseCollections(Enum):
    PENDING_JOBS = "PENDING_JOBS".lower()
    IN_PROGRESS_JOBS = "IN_PROGRESS_JOBS".lower()
    COMPLETED_JOBS = "COMPLETED_JOBS".lower()


class MultipleConnectorError(Exception):
    def __init__(self, message: str):
        self.message = message


class MongoDbConnector(DatabaseConnector):
    def __init__(self, connection_uri: str, database_id: str, connector_id: str = None):
        super().__init__(connection_uri, database_id, connector_id)

    def _get_client(self, *args):
        return MongoClient(args[0])

    def _get_database(self, db_id: str) -> Database:
        return self.client.get_database(db_id)

    def _get_jobs_from_collection(self, coll_name: str):
        return [job for job in self.db[coll_name].find()]
    
    @property
    def data(self):
        return self._get_data()
    
    def _get_data(self):
        return {coll_name: [v for v in self.db[coll_name].find()] for coll_name in self.db.list_collection_names()}
    
    def read(self, collection_name: DatabaseCollections | str, **kwargs):
        """Args:
            collection_name: str
            kwargs: (as in mongodb query)
        """
        coll_name = self._parse_enum_input(collection_name)
        coll = self.get_collection(coll_name)
        result = coll.find_one(kwargs.copy())
        return result

    def write(self, collection_name: DatabaseCollections | str, **kwargs):
        """
            Args:
                collection_name: str: collection name in mongodb
                **kwargs: mongo db `insert_one` query defining the document where the key is as in the key of the document.
        """
        coll_name = collection_name

        coll = self.get_collection(coll_name)
        result = coll.insert_one(kwargs.copy())
        return kwargs

    def get_collection(self, collection_name: str) -> Collection:
        try:
            return self.db[collection_name]
        except:
            return None

    async def insert_job_async(self, collection_name: str, **kwargs) -> Dict[str, Any]:
        return self.insert_job(collection_name, **kwargs)

    def insert_job(self, collection_name: str, **kwargs) -> Dict[str, Any]:
        coll = self.get_collection(collection_name)
        job_doc = kwargs.copy()
        print("Inserting job...")
        coll.insert_one(job_doc)
        print(f"Job successfully inserted: {self.db.pending_jobs.find_one(kwargs)}.")
        return kwargs

    async def update_job_status(self, collection_name: str, job_id: str, status: str | JobStatus):
        job_status = self._parse_enum_input(status)
        return self.db[collection_name].update_one({'job_id': job_id, }, {'$set': {'status': job_status}})

    def _parse_enum_input(self, _input: Any) -> str:
        return _input.value if isinstance(_input, Enum) else _input


In [6]:
from bson import ObjectId
{'_id': str(ObjectId('66df318e3c05234ee7df709b')),
   'job_id': 'e7619d85-c0a6-40df-9f30-e43e1a0a92fb',
   'last_updated': '2024-09-09 17:34:06.250744'}

In [8]:
uri = 'mongodb://localhost:27017/?retryWrites=true&w=majority&appName=biosimulators-processes'
conn = MongoDbConnector(connection_uri=uri, database_id="processes")

conn.data



In [32]:

import gridfs
import bson 
import json 

job_id = 'e7619d85-c0a6-40df-9f30-e43e1a0a92fb'

def format_data(conn):
    # goes to helpers
    formatted_data = {'process_results': []}
    for result in conn.data['process_results']:
        result['_id'] = str(result['_id'])
        formatted_data['process_results'].append(result)
    return formatted_data


def get_fs(conn):
    # goes to db connector
    return gridfs.GridFS(conn.db)   


def write_fs(conn, job_id):
    formatted_data = format_data(conn)
    fs = get_fs(conn)
    serialized_json = json.dumps(formatted_data)
    file_id = fs.put(serialized_json.encode('utf-8'), filename=job_id)
    return file_id


file_id = write_fs(conn, job_id)


def read_fs(conn, file_id):
    fs = get_fs(conn)
    stored_file = fs.get(file_id)
    retrieved_json = json.loads(stored_file.read().decode('utf-8'))
    return retrieved_json

In [33]:
file_id

In [31]:
read_fs(conn, file_id)

In [21]:
[j for j in conn.db['fs.files'].find()]

In [18]:
!ls

In [106]:
# run composite with copasi process and ram emitter
# on each update: return emitter val
# use db connector to write emitter val for update
# repeat


def run_composite_sse(duration, model_fp=model_fp, core=CORE):
    from process_bigraph import Composite 
    from uuid import uuid4

    uri = 'mongodb://localhost:27017/?retryWrites=true&w=majority&appName=biosimulators-processes'
    conn = MongoDbConnector(connection_uri=uri, database_id="processes")
    doc = {
        'copasi': {
            '_type': 'process',
            'address': 'local:copasi-process',
            'config': {
                'model': {
                    'model_source': model_fp
                }
            },
            'inputs': {
                'time': ['time_store'],
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'time': ['time_store'],
                'floating_species_concentrations': ['floating_species_concentrations_store'],
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'time': 'float',
                    'floating_species_concentrations': 'tree[float]'
                }
            },
            'inputs': {
                'time': ['time_store'],
                'floating_species_concentrations': ['floating_species_concentrations_store']
            }
        }
    }
    
    # make new job 
    job_id = str(uuid4())
    new_job = {'job_id': job_id}
    
    # insert mutable params
    write_job = new_job.copy()
    write_job.update({'last_updated': conn.timestamp(), 'data': []})
    job = conn.write(collection_name="process_results", **write_job)
    
    # make composite
    composite = Composite(config={'state': doc}, core=core)
    
    for n in range(duration):
        # run composite
        composite.run(1)
        
        # get historical results    
        results = composite.gather_results()
        data = results[('emitter',)]
        
        # find job and update data
        write_data = conn.db.process_results.find_one(new_job)
        write_data['data'] = data 
        
        # update db
        conn.db.process_results.update_one(new_job, {'$set': write_data})
        
    return conn

conn.refresh_data()
conn = run_composite_sse(10)

In [107]:
conn.data.get('process_results')

In [60]:
x = [{'a': list(range(10))}, {'a': list(range(10))}]

list(set(x))

In [103]:
s = {'a': 10}

s.update({'b': 3})

In [104]:
s

In [35]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/smoldyn/MinE.txt'
doc = {
        'process': {
            '_type': 'process',
            'address': 'local:smoldyn-process',
            'config': {
                'model': {
                    'model_source': model_fp
                }
            },
            "inputs": {
                "species_counts": [
                  "species_store"
                ],
                "molecules": [
                  "molecules_store"
                ]
              },
              "outputs": {
                "species_counts": [
                  "species_store"
                ],
                "molecules": [
                  "molecules_store"
                ]
              }
            },
            "emitter": {
              "_type": "step",
              "address": "local:database-emitter",
              "config": {
                "emit": {
                  "species_counts": "tree[integer]",
                  "molecules": "tree[string]"
                }
              },
              "inputs": {
                "species_counts": [
                  "species_store"
                ],
                "molecules": [
                  "molecules_store"
                ]
              }
            }
          }

from process_bigraph import Composite
from biosimulators_processes import CORE 
CORE.process_registry.registry

In [100]:
# THE goal is to be able to provide interval-wise smoldyn output files 

def read_smoldyn_simulation_configuration(filename):
    ''' Read a configuration for a Smoldyn simulation

    Args:
        filename (:obj:`str`): path to model file

    Returns:
        :obj:`list` of :obj:`str`: simulation configuration
    '''
    with open(filename, 'r') as file:
        return [line.strip('\n') for line in file]


def write_smoldyn_simulation_configuration(configuration, filename):
    ''' Write a configuration for Smoldyn simulation to a file

    Args:
        configuration
        filename (:obj:`str`): path to save configuration
    '''
    with open(filename, 'w') as file:
        for line in configuration:
            file.write(line)
            file.write('\n')
            


def add_output_commands(model_fp, duration):
    config = read_smoldyn_simulation_configuration(model_fp)
    has_output_commands = any([v.startswith('output') for v in config])
    if not has_output_commands:
        cmd_i = 0
        for i, v in enumerate(config):
            if v == 'end_file':
                cmd_i += i - 1
            stop_key = 'time_stop'
            
            if f'define {stop_key.upper()}' in v:
                new_v = f'define TIME_STOP   {duration}'
                config.remove(config[i])
                config.insert(i, new_v)
            elif v.startswith(stop_key):
                new_v = f'time_stop TIME_STOP'
                config.remove(config[i])
                config.insert(i, new_v)
        cmds = ["output_files modelout.txt",
                "cmd i 0 TIME_STOP 2 executiontime modelout.txt",
                "cmd i 0 TIME_STOP 2 listmols modelout.txt"]
        current = cmd_i 
        for cmd in cmds:
            config.insert(current, cmd)
            current += 1 
    
    write_smoldyn_simulation_configuration(config, model_fp) 
    
    out_file = model_fp.replace(model_fp.split('/')[-1].split('.')[0], 'modelout')
    return out_file
            
                
            
# in worker, get smoldyn config from bucket
# pass config through
add_output_commands(model_fp, 10)
        

In [94]:
with open(model_fp, 'w') as file:
    print(file.read())

In [56]:
x = list(range(20))

x.insert(2, [1, 2, 3])

In [2]:
comp = Composite(config={'state': doc}, core=CORE)

In [3]:
comp.run(3)


In [4]:
comp.gather_results()

In [1]:
import numpy as np
from tqdm import tqdm

from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import cobra
from cobra.io import load_model
model = load_model('textbook')

In [8]:
def add_dynamic_bounds(model, y):
    """Use external concentrations to bound the uptake flux of glucose."""
    biomass, glucose = y  # expand the boundary species
    glucose_max_import = -10 * glucose / (5 + glucose)
    model.reactions.EX_glc__D_e.lower_bound = glucose_max_import


def dynamic_system(t, y):
    """Calculate the time derivative of external species."""

    biomass, glucose = y  # expand the boundary species

    # Calculate the specific exchanges fluxes at the given external concentrations.
    with model:
        add_dynamic_bounds(model, y)

        cobra.util.add_lp_feasibility(model)
        feasibility = cobra.util.fix_objective_as_constraint(model)
        lex_constraints = cobra.util.add_lexicographic_constraints(
            model=model, 
            objectives=['Biomass_Ecoli_core', 'EX_glc__D_e'], 
            objective_direction=['max', 'max']
        )

    # Since the calculated fluxes are specific rates, we multiply them by the
    # biomass concentration to get the bulk exchange rates.
    fluxes = lex_constraints.values
    fluxes *= biomass

    # This implementation is **not** efficient, so I display the current
    # simulation time using a progress bar.
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes

dynamic_system.pbar = None


def infeasible_event(t, y):
    """
    Determine solution feasibility.

    Avoiding infeasible solutions is handled by solve_ivp's built-in event detection.
    This function re-solves the LP to determine whether or not the solution is feasible
    (and if not, how far it is from feasibility). When the sign of this function changes
    from -epsilon to positive, we know the solution is no longer feasible.

    """

    with model:

        add_dynamic_bounds(model, y)

        cobra.util.add_lp_feasibility(model)
        feasibility = cobra.util.fix_objective_as_constraint(model)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [9]:
ts = np.linspace(0, 15, 100)  # Desired integration resolution and interval
y0 = [0.1, 10]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )

In [10]:
sol